⚡ Avancé | ⏱ 60 min | 🔑 Concepts : delta time, vélocité, gravité, rebonds, interpolation

# Animations et Physique

## Objectifs

- Maîtriser le delta time pour un mouvement indépendant des FPS
- Implémenter la vélocité et l'accélération avec des vecteurs
- Simuler la gravité et les sauts
- Gérer les rebonds et la friction
- Créer des animations par spritesheet
- Utiliser l'interpolation et les fonctions d'easing

## Prérequis

- Notions de base de Pygame (notebooks 01-05)
- Compréhension des sprites et groupes
- Notions de base en physique (vitesse, accélération)

## 1. Delta Time Approfondi

### 1.1 Pourquoi clock.tick() ne suffit pas

`clock.tick(60)` limite la boucle à 60 FPS, mais ne **garantit** pas 60 FPS. Si le jeu ralentit (beaucoup d'objets, calculs lourds), le FPS réel peut baisser.

```python
# Problème : sans delta time, le mouvement dépend des FPS
# À 60 FPS : 5 * 60 = 300 pixels/seconde
# À 30 FPS : 5 * 30 = 150 pixels/seconde  <- Deux fois plus lent!
x += 5  # Pixels par frame
```

### 1.2 Calcul du delta time

```python
clock = pygame.time.Clock()

while running:
    # dt = temps écoulé depuis le dernier frame, en secondes
    dt = clock.tick(60) / 1000.0
    
    # À 60 FPS : dt ≈ 0.0167 s
    # À 30 FPS : dt ≈ 0.0333 s
    
    # Mouvement constant quelle que soit la vitesse
    vitesse = 300  # pixels par seconde
    x += vitesse * dt  # 300 * 0.0167 ≈ 5 pixels à 60 FPS
                       # 300 * 0.0333 ≈ 10 pixels à 30 FPS
                       # Résultat : même distance par seconde!
```

### 1.3 Limiter le delta time

Si le jeu freeze un instant (chargement, alt-tab), dt peut être très grand, causant des "téléportations" :

```python
dt = clock.tick(60) / 1000.0
dt = min(dt, 0.05)  # Maximum 50ms (équivalent à 20 FPS minimum)
```

## 2. Vélocité et Accélération

### 2.1 Vecteurs de mouvement

Un objet en mouvement a une **position** et une **vélocité** (vitesse avec direction) :

```python
# Sans Vector2 (manuel)
pos_x, pos_y = 400, 300
vel_x, vel_y = 100, -50  # pixels/seconde

# Mise à jour
pos_x += vel_x * dt
pos_y += vel_y * dt
```

### 2.2 pygame.math.Vector2

Pygame fournit une classe `Vector2` pour simplifier les calculs vectoriels :

```python
from pygame.math import Vector2

# Créer des vecteurs
position = Vector2(400, 300)
velocite = Vector2(100, -50)

# Opérations
position += velocite * dt          # Déplacement
distance = position.distance_to(cible)  # Distance
direction = velocite.normalize()   # Direction unitaire
vitesse = velocite.length()        # Vitesse scalaire
angle = velocite.angle_to(Vector2(1, 0))  # Angle

# Rotation
v_tourne = velocite.rotate(45)     # Tourner de 45°

# Accès aux composantes
print(position.x, position.y)
```

### 2.3 Accélération

L'accélération modifie la vélocité dans le temps :

```python
position = Vector2(400, 300)
velocite = Vector2(0, 0)
acceleration = Vector2(0, 0)

# Dans la boucle
# 1. Appliquer les forces (clavier, gravité...)
acceleration = Vector2(0, 0)
if keys[pygame.K_RIGHT]:
    acceleration.x = 500  # pixels/s²

# 2. Intégrer l'accélération dans la vélocité
velocite += acceleration * dt

# 3. Intégrer la vélocité dans la position
position += velocite * dt
```

## 3. Gravité et Sauts

### 3.1 Simuler la gravité

La gravité est une accélération constante vers le bas :

```python
GRAVITE = 980  # pixels/s² (approximation de 9.8 m/s² mise à l'échelle)

vel_y = 0
pos_y = 100

# Dans la boucle
vel_y += GRAVITE * dt  # La vélocité augmente
pos_y += vel_y * dt     # La position change
```

### 3.2 Détection du sol

```python
SOL = 500  # Position Y du sol

if pos_y >= SOL:
    pos_y = SOL
    vel_y = 0
    au_sol = True
else:
    au_sol = False
```

### 3.3 Mécanique de saut

```python
FORCE_SAUT = -500  # Négatif = vers le haut

# Saut simple
if event.type == pygame.KEYDOWN:
    if event.key == pygame.K_SPACE and au_sol:
        vel_y = FORCE_SAUT
        au_sol = False
```

### 3.4 Saut variable (maintenir pour sauter plus haut)

```python
# En relâchant la touche, réduire la vélocité vers le haut
if event.type == pygame.KEYUP:
    if event.key == pygame.K_SPACE and vel_y < 0:
        vel_y *= 0.5  # Couper la montée
```

### 3.5 Double saut

```python
nb_sauts = 0
MAX_SAUTS = 2

if event.key == pygame.K_SPACE and nb_sauts < MAX_SAUTS:
    vel_y = FORCE_SAUT
    nb_sauts += 1

# Réinitialiser au sol
if au_sol:
    nb_sauts = 0
```

## 4. Rebonds et Friction

### 4.1 Rebond élastique

Un rebond parfait inverse la composante de la vélocité :

```python
# Rebond horizontal
if pos_x <= 0 or pos_x >= LARGEUR:
    vel_x = -vel_x  # Inverser la direction

# Rebond vertical
if pos_y <= 0 or pos_y >= HAUTEUR:
    vel_y = -vel_y
```

### 4.2 Rebond inélastique (avec perte d'énergie)

```python
COEFFICIENT_REBOND = 0.7  # 0 = pas de rebond, 1 = rebond parfait

if pos_y >= SOL:
    pos_y = SOL
    vel_y = -vel_y * COEFFICIENT_REBOND  # Perd 30% de l'énergie
    
    # Arrêter le rebond quand la vélocité est trop faible
    if abs(vel_y) < 10:
        vel_y = 0
```

### 4.3 Friction / Amortissement

La friction ralentit progressivement un objet :

```python
FRICTION = 0.98  # Coefficient de friction (0.95 à 0.99)

# Appliquer la friction à chaque frame
vel_x *= FRICTION
vel_y *= FRICTION

# Version indépendante du framerate
vel_x *= FRICTION ** (dt * 60)  # Normalisé pour 60 FPS
```

### 4.4 Friction au sol uniquement

```python
if au_sol:
    vel_x *= 0.85  # Forte friction au sol
else:
    vel_x *= 0.99  # Faible résistance de l'air
```

## 5. Animation par Spritesheet

### 5.1 Principe

Un **spritesheet** est une image contenant toutes les frames d'une animation. On affiche les frames une par une.

```
┌──────┬──────┬──────┬──────┐
│ Frame│ Frame│ Frame│ Frame│
│  0   │  1   │  2   │  3   │
└──────┴──────┴──────┴──────┘
```

### 5.2 Découper un spritesheet

```python
def charger_spritesheet(chemin, largeur_frame, hauteur_frame, nb_frames):
    """Découper un spritesheet en frames individuelles"""
    spritesheet = pygame.image.load(chemin).convert_alpha()
    frames = []
    
    for i in range(nb_frames):
        x = i * largeur_frame
        frame = spritesheet.subsurface((x, 0, largeur_frame, hauteur_frame))
        frames.append(frame)
    
    return frames
```

### 5.3 Animation basée sur un timer

```python
class SpriteAnime(pygame.sprite.Sprite):
    def __init__(self, frames, vitesse_anim=0.1):
        super().__init__()
        self.frames = frames
        self.index = 0
        self.timer = 0
        self.vitesse_anim = vitesse_anim  # Secondes entre chaque frame
        self.image = self.frames[0]
        self.rect = self.image.get_rect()
    
    def update(self, dt):
        self.timer += dt
        if self.timer >= self.vitesse_anim:
            self.timer -= self.vitesse_anim
            self.index = (self.index + 1) % len(self.frames)
            self.image = self.frames[self.index]
```

### 5.4 Machine à états d'animation

```python
class Personnage(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        # Animations par état
        self.animations = {
            'idle': [...],   # Frames d'attente
            'walk': [...],   # Frames de marche
            'jump': [...],   # Frames de saut
        }
        self.etat = 'idle'
        self.index = 0
        self.timer = 0
    
    def changer_etat(self, nouvel_etat):
        if self.etat != nouvel_etat:
            self.etat = nouvel_etat
            self.index = 0
            self.timer = 0
    
    def update(self, dt):
        # Déterminer l'état
        if not self.au_sol:
            self.changer_etat('jump')
        elif abs(self.vel_x) > 10:
            self.changer_etat('walk')
        else:
            self.changer_etat('idle')
        
        # Animer
        frames = self.animations[self.etat]
        self.timer += dt
        if self.timer >= 0.1:
            self.timer -= 0.1
            self.index = (self.index + 1) % len(frames)
        self.image = frames[self.index]
```

## 6. Interpolation et Easing

### 6.1 Interpolation linéaire (lerp)

L'interpolation linéaire calcule une valeur entre deux points :

```python
def lerp(a, b, t):
    """Interpolation linéaire : a quand t=0, b quand t=1"""
    return a + (b - a) * t

# Exemple : déplacer un objet de 100 à 500 en 2 secondes
temps_ecoule = 0
duree = 2.0

temps_ecoule += dt
t = min(temps_ecoule / duree, 1.0)  # 0 à 1
x = lerp(100, 500, t)
```

### 6.2 Fonctions d'easing

Les fonctions d'easing rendent les mouvements plus naturels :

```python
import math

def ease_in_quad(t):
    """Démarrage lent, accélère"""
    return t * t

def ease_out_quad(t):
    """Démarrage rapide, décélère"""
    return 1 - (1 - t) ** 2

def ease_in_out_quad(t):
    """Lent au début et à la fin"""
    if t < 0.5:
        return 2 * t * t
    else:
        return 1 - (-2 * t + 2) ** 2 / 2

def ease_out_bounce(t):
    """Effet de rebond"""
    if t < 1/2.75:
        return 7.5625 * t * t
    elif t < 2/2.75:
        t -= 1.5/2.75
        return 7.5625 * t * t + 0.75
    elif t < 2.5/2.75:
        t -= 2.25/2.75
        return 7.5625 * t * t + 0.9375
    else:
        t -= 2.625/2.75
        return 7.5625 * t * t + 0.984375

def ease_out_elastic(t):
    """Effet élastique"""
    if t == 0 or t == 1:
        return t
    return 2 ** (-10 * t) * math.sin((t - 0.075) * (2 * math.pi) / 0.3) + 1

def ease_in_out_sine(t):
    """Transition douce sinusoïdale"""
    return -(math.cos(math.pi * t) - 1) / 2
```

### 6.3 Utilisation avec lerp

```python
# Au lieu de lerp linéaire
t = temps_ecoule / duree
t_ease = ease_out_quad(t)  # Appliquer l'easing
x = lerp(depart, arrivee, t_ease)
```

In [ ]:
%%writefile demo_gravite.py
"""Démonstration : Balles avec gravité et rebonds
Cliquez pour lancer des balles qui tombent et rebondissent.
"""
import pygame
import sys
import random
import math
from pygame.math import Vector2

pygame.init()

LARGEUR, HAUTEUR = 900, 650
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Gravité et Rebonds")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

# Constantes physiques
GRAVITE = 800        # pixels/s²
COEFF_REBOND = 0.75  # Coefficient de restitution
FRICTION_SOL = 0.92  # Friction au sol
RESISTANCE_AIR = 0.999  # Résistance de l'air
SOL_Y = HAUTEUR - 50

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (80, 80, 80)


class Balle:
    """Balle soumise à la gravité"""
    def __init__(self, x, y, vx=0, vy=0):
        self.pos = Vector2(x, y)
        self.vel = Vector2(vx, vy)
        self.rayon = random.randint(10, 25)
        self.couleur = (
            random.randint(100, 255),
            random.randint(100, 255),
            random.randint(100, 255)
        )
        self.active = True
        self.trace = []  # Traînée visuelle
    
    def update(self, dt):
        # Sauvegarder la position pour la traînée
        if len(self.trace) > 20:
            self.trace.pop(0)
        self.trace.append(Vector2(self.pos))
        
        # Appliquer la gravité
        self.vel.y += GRAVITE * dt
        
        # Résistance de l'air
        self.vel *= RESISTANCE_AIR
        
        # Déplacer
        self.pos += self.vel * dt
        
        # Rebond sur le sol
        if self.pos.y + self.rayon >= SOL_Y:
            self.pos.y = SOL_Y - self.rayon
            self.vel.y = -abs(self.vel.y) * COEFF_REBOND
            self.vel.x *= FRICTION_SOL
            
            # Arrêter si trop lent
            if abs(self.vel.y) < 15:
                self.vel.y = 0
                self.pos.y = SOL_Y - self.rayon
        
        # Rebond sur les murs
        if self.pos.x - self.rayon < 0:
            self.pos.x = self.rayon
            self.vel.x = abs(self.vel.x) * COEFF_REBOND
        elif self.pos.x + self.rayon > LARGEUR:
            self.pos.x = LARGEUR - self.rayon
            self.vel.x = -abs(self.vel.x) * COEFF_REBOND
        
        # Rebond sur le plafond
        if self.pos.y - self.rayon < 0:
            self.pos.y = self.rayon
            self.vel.y = abs(self.vel.y) * COEFF_REBOND
        
        # Désactiver si immobile
        if (abs(self.vel.x) < 1 and abs(self.vel.y) < 1 and
                self.pos.y + self.rayon >= SOL_Y - 2):
            self.vel = Vector2(0, 0)
    
    def dessiner(self, surface):
        # Dessiner la traînée
        for i, pos in enumerate(self.trace):
            alpha = i / len(self.trace) if self.trace else 0
            rayon = max(2, int(self.rayon * alpha * 0.5))
            couleur = tuple(int(c * alpha * 0.4) for c in self.couleur)
            pygame.draw.circle(surface, couleur, (int(pos.x), int(pos.y)), rayon)
        
        # Dessiner la balle
        pygame.draw.circle(surface, self.couleur,
                          (int(self.pos.x), int(self.pos.y)), self.rayon)
        # Reflet
        reflet_pos = (int(self.pos.x - self.rayon * 0.3),
                      int(self.pos.y - self.rayon * 0.3))
        pygame.draw.circle(surface, BLANC, reflet_pos, max(2, self.rayon // 4))


# Liste des balles
balles = []

# Variables d'interaction
lancer_en_cours = False
point_depart = None

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(60) / 1000.0
    dt = min(dt, 0.05)  # Limiter le delta time
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_c:
                balles.clear()
        
        # Système de lancer : clic et glisser
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                lancer_en_cours = True
                point_depart = Vector2(event.pos)
        
        if event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1 and lancer_en_cours:
                point_fin = Vector2(event.pos)
                # La vélocité est proportionnelle au glissement
                vel = (point_depart - point_fin) * 3
                balle = Balle(point_depart.x, point_depart.y, vel.x, vel.y)
                balles.append(balle)
                lancer_en_cours = False
    
    # Mise à jour de toutes les balles
    for balle in balles:
        balle.update(dt)
    
    # Limiter le nombre de balles
    if len(balles) > 100:
        balles = balles[-100:]
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Sol
    pygame.draw.line(ecran, GRIS, (0, SOL_Y), (LARGEUR, SOL_Y), 2)
    pygame.draw.rect(ecran, (30, 30, 30), (0, SOL_Y, LARGEUR, HAUTEUR - SOL_Y))
    
    # Dessiner les balles
    for balle in balles:
        balle.dessiner(ecran)
    
    # Ligne de lancer
    if lancer_en_cours and point_depart:
        mouse_pos = pygame.mouse.get_pos()
        pygame.draw.line(ecran, (255, 100, 100),
                        (int(point_depart.x), int(point_depart.y)),
                        mouse_pos, 2)
        # Flèche de direction
        direction = point_depart - Vector2(mouse_pos)
        if direction.length() > 0:
            force_txt = font.render(f"Force: {int(direction.length())}", True, BLANC)
            ecran.blit(force_txt, (int(point_depart.x) + 10, int(point_depart.y) - 25))
    
    # Interface
    info1 = font.render(f"Balles: {len(balles)} | FPS: {clock.get_fps():.0f}", True, BLANC)
    info2 = font.render("Clic + glisser: lancer | C: effacer", True, BLANC)
    info3 = font.render(f"Gravité: {GRAVITE} | Rebond: {COEFF_REBOND}", True, GRIS)
    ecran.blit(info1, (10, 10))
    ecran.blit(info2, (10, 38))
    ecran.blit(info3, (10, HAUTEUR - 25))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

In [ ]:
%%writefile demo_animation.py
"""Démonstration : Personnage animé avec états idle/walk/jump
Utilise des frames procédurales (pas besoin de fichiers images).
"""
import pygame
import sys
import math
from pygame.math import Vector2

pygame.init()

LARGEUR, HAUTEUR = 800, 500
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Animation par états - Flèches + Espace")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

# Constantes
GRAVITE = 1200
SOL_Y = HAUTEUR - 80
FORCE_SAUT = -550

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
VERT = (80, 200, 80)
BLEU = (80, 120, 255)
MARRON = (139, 90, 43)


def creer_frame_personnage(largeur, hauteur, couleur_corps, pose="normal"):
    """Créer une frame d'animation procédurale pour le personnage"""
    surface = pygame.Surface((largeur, hauteur), pygame.SRCALPHA)
    cx = largeur // 2
    
    # Tête
    pygame.draw.circle(surface, couleur_corps, (cx, 12), 10)
    
    # Corps
    pygame.draw.rect(surface, couleur_corps, (cx - 8, 22, 16, 20))
    
    # Jambes selon la pose
    if pose == "normal":
        pygame.draw.rect(surface, couleur_corps, (cx - 7, 42, 6, 16))
        pygame.draw.rect(surface, couleur_corps, (cx + 1, 42, 6, 16))
    elif pose == "walk1":
        pygame.draw.line(surface, couleur_corps, (cx - 3, 42), (cx - 10, 58), 5)
        pygame.draw.line(surface, couleur_corps, (cx + 3, 42), (cx + 10, 58), 5)
    elif pose == "walk2":
        pygame.draw.line(surface, couleur_corps, (cx - 3, 42), (cx + 2, 58), 5)
        pygame.draw.line(surface, couleur_corps, (cx + 3, 42), (cx - 2, 58), 5)
    elif pose == "jump":
        pygame.draw.line(surface, couleur_corps, (cx - 3, 42), (cx - 12, 52), 5)
        pygame.draw.line(surface, couleur_corps, (cx + 3, 42), (cx + 12, 52), 5)
    
    # Bras
    if pose == "jump":
        pygame.draw.line(surface, couleur_corps, (cx - 8, 26), (cx - 18, 16), 4)
        pygame.draw.line(surface, couleur_corps, (cx + 8, 26), (cx + 18, 16), 4)
    elif pose == "walk1":
        pygame.draw.line(surface, couleur_corps, (cx - 8, 26), (cx + 4, 38), 4)
        pygame.draw.line(surface, couleur_corps, (cx + 8, 26), (cx - 4, 38), 4)
    elif pose == "walk2":
        pygame.draw.line(surface, couleur_corps, (cx - 8, 26), (cx - 4, 38), 4)
        pygame.draw.line(surface, couleur_corps, (cx + 8, 26), (cx + 4, 38), 4)
    else:
        pygame.draw.line(surface, couleur_corps, (cx - 8, 26), (cx - 14, 38), 4)
        pygame.draw.line(surface, couleur_corps, (cx + 8, 26), (cx + 14, 38), 4)
    
    # Yeux
    pygame.draw.circle(surface, BLANC, (cx - 4, 10), 3)
    pygame.draw.circle(surface, BLANC, (cx + 4, 10), 3)
    pygame.draw.circle(surface, NOIR, (cx - 3, 10), 1)
    pygame.draw.circle(surface, NOIR, (cx + 5, 10), 1)
    
    return surface


# Créer les animations
TAILLE_PERSO = (40, 60)
couleur = BLEU

animations = {
    'idle': [
        creer_frame_personnage(*TAILLE_PERSO, couleur, "normal"),
        creer_frame_personnage(*TAILLE_PERSO, couleur, "normal"),
    ],
    'walk': [
        creer_frame_personnage(*TAILLE_PERSO, couleur, "normal"),
        creer_frame_personnage(*TAILLE_PERSO, couleur, "walk1"),
        creer_frame_personnage(*TAILLE_PERSO, couleur, "normal"),
        creer_frame_personnage(*TAILLE_PERSO, couleur, "walk2"),
    ],
    'jump': [
        creer_frame_personnage(*TAILLE_PERSO, couleur, "jump"),
    ],
}

# État du personnage
pos = Vector2(LARGEUR // 2, SOL_Y - TAILLE_PERSO[1])
vel = Vector2(0, 0)
etat = 'idle'
index_frame = 0
timer_anim = 0
vitesse_anim = 0.12  # Secondes entre frames
regarde_droite = True
au_sol = True
nb_sauts = 0
MAX_SAUTS = 2

# Accélération du joueur
ACCEL = 1200
VITESSE_MAX = 350
FRICTION = 0.85

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(60) / 1000.0
    dt = min(dt, 0.05)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            # Saut
            if event.key == pygame.K_SPACE and nb_sauts < MAX_SAUTS:
                vel.y = FORCE_SAUT
                nb_sauts += 1
                au_sol = False
        # Saut variable
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE and vel.y < 0:
                vel.y *= 0.5
    
    # ---- Mouvement ----
    keys = pygame.key.get_pressed()
    
    # Accélération horizontale
    if keys[pygame.K_LEFT]:
        vel.x -= ACCEL * dt
        regarde_droite = False
    elif keys[pygame.K_RIGHT]:
        vel.x += ACCEL * dt
        regarde_droite = True
    else:
        # Friction quand aucune touche n'est enfoncée
        vel.x *= FRICTION
    
    # Limiter la vitesse horizontale
    vel.x = max(-VITESSE_MAX, min(VITESSE_MAX, vel.x))
    
    # Gravité
    vel.y += GRAVITE * dt
    
    # Déplacer
    pos += vel * dt
    
    # Sol
    if pos.y + TAILLE_PERSO[1] >= SOL_Y:
        pos.y = SOL_Y - TAILLE_PERSO[1]
        vel.y = 0
        au_sol = True
        nb_sauts = 0
    else:
        au_sol = False
    
    # Limiter aux bords
    pos.x = max(0, min(pos.x, LARGEUR - TAILLE_PERSO[0]))
    
    # ---- Animation ----
    # Déterminer l'état
    ancien_etat = etat
    if not au_sol:
        etat = 'jump'
    elif abs(vel.x) > 20:
        etat = 'walk'
    else:
        etat = 'idle'
    
    # Réinitialiser l'animation si changement d'état
    if etat != ancien_etat:
        index_frame = 0
        timer_anim = 0
    
    # Avancer l'animation
    timer_anim += dt
    vitesse_courante = vitesse_anim
    if etat == 'walk':
        # Animation plus rapide quand le personnage va vite
        vitesse_courante = max(0.05, 0.15 - abs(vel.x) / 3000)
    
    if timer_anim >= vitesse_courante:
        timer_anim -= vitesse_courante
        frames = animations[etat]
        index_frame = (index_frame + 1) % len(frames)
    
    # ---- Affichage ----
    ecran.fill((20, 20, 40))
    
    # Sol
    pygame.draw.line(ecran, VERT, (0, SOL_Y), (LARGEUR, SOL_Y), 3)
    pygame.draw.rect(ecran, (40, 100, 40), (0, SOL_Y, LARGEUR, HAUTEUR - SOL_Y))
    
    # Personnage
    frames = animations[etat]
    image = frames[index_frame % len(frames)]
    if not regarde_droite:
        image = pygame.transform.flip(image, True, False)
    ecran.blit(image, (int(pos.x), int(pos.y)))
    
    # Ombre au sol
    hauteur_vol = SOL_Y - (pos.y + TAILLE_PERSO[1])
    ombre_largeur = max(10, TAILLE_PERSO[0] - int(hauteur_vol * 0.1))
    ombre_alpha = max(30, 150 - int(hauteur_vol * 0.5))
    ombre_surface = pygame.Surface((ombre_largeur, 6), pygame.SRCALPHA)
    pygame.draw.ellipse(ombre_surface, (0, 0, 0, ombre_alpha),
                       (0, 0, ombre_largeur, 6))
    ecran.blit(ombre_surface,
              (int(pos.x + TAILLE_PERSO[0] // 2 - ombre_largeur // 2), SOL_Y - 3))
    
    # Interface
    etat_txt = font.render(f"État: {etat} | Frame: {index_frame}", True, BLANC)
    vel_txt = font.render(f"Vélocité: ({vel.x:.0f}, {vel.y:.0f})", True, BLANC)
    saut_txt = font.render(f"Sauts: {nb_sauts}/{MAX_SAUTS}", True, BLANC)
    ctrl_txt = font.render("Flèches: bouger | ESPACE: sauter (x2)", True, (150, 150, 150))
    
    ecran.blit(etat_txt, (10, 10))
    ecran.blit(vel_txt, (10, 38))
    ecran.blit(saut_txt, (10, 66))
    ecran.blit(ctrl_txt, (10, HAUTEUR - 30))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

In [ ]:
%%writefile demo_particules.py
"""Démonstration : Système de particules
Cliquez pour créer des explosions de particules.
"""
import pygame
import sys
import random
import math
from pygame.math import Vector2

pygame.init()

LARGEUR, HAUTEUR = 900, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Système de Particules")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)


class Particule:
    """Une particule avec position, vélocité et durée de vie"""
    def __init__(self, x, y, couleur_base, type_part="explosion"):
        self.pos = Vector2(x, y)
        self.type = type_part
        
        if type_part == "explosion":
            angle = random.uniform(0, 2 * math.pi)
            vitesse = random.uniform(50, 350)
            self.vel = Vector2(math.cos(angle) * vitesse, math.sin(angle) * vitesse)
            self.rayon = random.uniform(2, 6)
            self.duree_vie = random.uniform(0.5, 1.5)
            self.gravite = 200
        elif type_part == "etincelle":
            angle = random.uniform(-math.pi * 0.8, -math.pi * 0.2)
            vitesse = random.uniform(100, 300)
            self.vel = Vector2(math.cos(angle) * vitesse, math.sin(angle) * vitesse)
            self.rayon = random.uniform(1, 3)
            self.duree_vie = random.uniform(0.3, 0.8)
            self.gravite = 400
        elif type_part == "fumee":
            self.vel = Vector2(random.uniform(-20, 20), random.uniform(-80, -30))
            self.rayon = random.uniform(5, 15)
            self.duree_vie = random.uniform(1.0, 3.0)
            self.gravite = -10  # Monte lentement
        elif type_part == "fontaine":
            angle = random.uniform(-math.pi/2 - 0.3, -math.pi/2 + 0.3)
            vitesse = random.uniform(200, 400)
            self.vel = Vector2(math.cos(angle) * vitesse, math.sin(angle) * vitesse)
            self.rayon = random.uniform(2, 5)
            self.duree_vie = random.uniform(1.0, 2.5)
            self.gravite = 300
        
        self.temps_restant = self.duree_vie
        self.couleur_base = couleur_base
        self.vivant = True
    
    def update(self, dt):
        # Gravité
        self.vel.y += self.gravite * dt
        
        # Déplacer
        self.pos += self.vel * dt
        
        # Durée de vie
        self.temps_restant -= dt
        if self.temps_restant <= 0:
            self.vivant = False
        
        # Réduire la taille pour la fumée
        if self.type == "fumee":
            ratio = self.temps_restant / self.duree_vie
            self.rayon = max(1, self.rayon * (0.98 + 0.02 * ratio))
    
    def dessiner(self, surface):
        ratio = max(0, self.temps_restant / self.duree_vie)
        
        # Couleur qui s'estompe
        r = min(255, int(self.couleur_base[0] * ratio))
        g = min(255, int(self.couleur_base[1] * ratio))
        b = min(255, int(self.couleur_base[2] * ratio))
        couleur = (r, g, b)
        
        # Taille qui diminue
        rayon = max(1, int(self.rayon * ratio))
        
        pygame.draw.circle(surface, couleur,
                          (int(self.pos.x), int(self.pos.y)), rayon)


class SystemeParticules:
    """Gère toutes les particules"""
    def __init__(self):
        self.particules = []
    
    def explosion(self, x, y, nb=50, couleur=(255, 150, 50)):
        for _ in range(nb):
            self.particules.append(Particule(x, y, couleur, "explosion"))
    
    def etincelles(self, x, y, nb=20, couleur=(255, 255, 100)):
        for _ in range(nb):
            self.particules.append(Particule(x, y, couleur, "etincelle"))
    
    def fumee(self, x, y, nb=5, couleur=(150, 150, 150)):
        for _ in range(nb):
            self.particules.append(Particule(x, y, couleur, "fumee"))
    
    def fontaine(self, x, y, nb=3, couleur=(80, 150, 255)):
        for _ in range(nb):
            self.particules.append(Particule(x, y, couleur, "fontaine"))
    
    def update(self, dt):
        for p in self.particules:
            p.update(dt)
        # Retirer les particules mortes
        self.particules = [p for p in self.particules if p.vivant]
    
    def dessiner(self, surface):
        for p in self.particules:
            p.dessiner(surface)


# Système de particules
systeme = SystemeParticules()

# Mode actuel
modes = ["explosion", "etincelles", "fumee", "fontaine"]
couleurs_modes = [
    (255, 150, 50),    # Explosion : orange
    (255, 255, 100),   # Étincelles : jaune
    (150, 150, 150),   # Fumée : gris
    (80, 150, 255),    # Fontaine : bleu
]
index_mode = 0

# Fontaine continue
fontaine_active = False
fontaine_pos = Vector2(LARGEUR // 2, HAUTEUR - 50)

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(60) / 1000.0
    dt = min(dt, 0.05)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_TAB:
                index_mode = (index_mode + 1) % len(modes)
            if event.key == pygame.K_c:
                systeme.particules.clear()
            if event.key == pygame.K_f:
                fontaine_active = not fontaine_active
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            mode = modes[index_mode]
            couleur = couleurs_modes[index_mode]
            
            if mode == "explosion":
                systeme.explosion(mx, my, 60, couleur)
            elif mode == "etincelles":
                systeme.etincelles(mx, my, 30, couleur)
            elif mode == "fumee":
                systeme.fumee(mx, my, 15, couleur)
            elif mode == "fontaine":
                fontaine_pos = Vector2(mx, my)
                fontaine_active = True
    
    # Fontaine continue
    if fontaine_active:
        systeme.fontaine(fontaine_pos.x, fontaine_pos.y, 5,
                        couleurs_modes[3])
    
    # Mise à jour
    systeme.update(dt)
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Dessiner les particules
    systeme.dessiner(ecran)
    
    # Interface
    mode_txt = font.render(
        f"Mode: {modes[index_mode].upper()} (TAB pour changer)",
        True, couleurs_modes[index_mode]
    )
    nb_txt = font.render(
        f"Particules: {len(systeme.particules)} | FPS: {clock.get_fps():.0f}",
        True, BLANC
    )
    ctrl_txt = font.render(
        "Clic: créer | F: fontaine on/off | C: effacer",
        True, (150, 150, 150)
    )
    
    ecran.blit(mode_txt, (10, 10))
    ecran.blit(nb_txt, (10, 38))
    ecran.blit(ctrl_txt, (10, HAUTEUR - 30))
    
    if fontaine_active:
        font_txt = font.render("Fontaine: ON", True, (80, 150, 255))
        ecran.blit(font_txt, (LARGEUR - 160, 10))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

## Pièges Courants

### 1. Ne pas limiter le delta time

```python
# ❌ ERREUR : si le jeu freeze 2s, dt = 2.0 et l'objet traverse les murs
dt = clock.tick(60) / 1000.0
pos.x += vel.x * dt  # Si dt = 2.0, déplacement énorme!

# ✅ CORRECT : limiter dt à une valeur maximale
dt = clock.tick(60) / 1000.0
dt = min(dt, 0.05)  # Maximum 50ms
pos.x += vel.x * dt
```

### 2. Friction qui dépend des FPS

```python
# ❌ ERREUR : la friction est plus forte à 60 FPS qu'à 30 FPS
vel.x *= 0.95  # Appliqué par frame!

# ✅ CORRECT : normaliser la friction pour le delta time
vel.x *= 0.95 ** (dt * 60)  # Résultat constant quel que soit le FPS
```

### 3. Oublier de réinitialiser l'accélération

```python
# ❌ ERREUR : l'accélération s'accumule!
if keys[pygame.K_RIGHT]:
    acceleration.x += 500 * dt  # S'accumule chaque frame!

# ✅ CORRECT : réinitialiser à chaque frame
acceleration = Vector2(0, 0)  # Reset!
if keys[pygame.K_RIGHT]:
    acceleration.x = 500
vel += acceleration * dt
```

### 4. Rotation d'animation cumulative

```python
# ❌ ERREUR : la qualité se dégrade à chaque rotation
image = pygame.transform.rotate(image, 5)  # Cumulative!

# ✅ CORRECT : toujours tourner depuis l'original
angle += 5
image_affichee = pygame.transform.rotate(image_originale, angle)
```

### 5. Saut infini

```python
# ❌ ERREUR : on peut sauter indéfiniment
if event.key == pygame.K_SPACE:
    vel_y = FORCE_SAUT  # Pas de vérification!

# ✅ CORRECT : vérifier qu'on est au sol
if event.key == pygame.K_SPACE and au_sol:
    vel_y = FORCE_SAUT
    au_sol = False
```

## Mini-Exercices

### Exercice 1 : Balle avec gravité et rebonds

Créez un programme avec :
- Une balle qui tombe sous l'effet de la gravité
- Rebonds inélastiques (perd de l'énergie à chaque rebond)
- Clic pour repositionner la balle
- Afficher la vélocité et la hauteur
- La balle finit par s'arrêter au sol

In [ ]:
# Exercice 1 : À vous de coder!


### Exercice 2 : Platformer basique

Créez un mini-platformer avec :
- Un personnage qui se déplace avec les flèches
- Saut avec ESPACE (avec gravité)
- Double saut
- Friction au sol
- 3 plateformes à différentes hauteurs (rectangles)
- Détection de collision avec les plateformes

In [ ]:
# Exercice 2 : À vous de coder!


### Exercice 3 : Animation avec easing

Créez un programme qui :
- Affiche un carré qui se déplace d'un point A à un point B
- Touches 1-5 pour changer la fonction d'easing (linear, ease_in, ease_out, bounce, elastic)
- ESPACE pour relancer l'animation
- Afficher le nom de l'easing et la progression
- Tracer la courbe d'easing en bas de l'écran

In [ ]:
# Exercice 3 : À vous de coder!


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile solution_ex1_gravite.py
import pygame
import sys
from pygame.math import Vector2

pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 1 : Gravité et rebonds")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 32)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
ROUGE = (255, 80, 80)
GRIS = (80, 80, 80)

# Physique
GRAVITE = 800
REBOND = 0.7
SOL = HAUTEUR - 50

# Balle
pos = Vector2(LARGEUR // 2, 100)
vel = Vector2(150, 0)
rayon = 20

running = True
while running:
    dt = clock.tick(60) / 1000.0
    dt = min(dt, 0.05)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            pos = Vector2(event.pos)
            vel = Vector2(0, 0)
    
    # Gravité
    vel.y += GRAVITE * dt
    pos += vel * dt
    
    # Rebond sol
    if pos.y + rayon >= SOL:
        pos.y = SOL - rayon
        vel.y = -abs(vel.y) * REBOND
        vel.x *= 0.95  # Friction sol
        if abs(vel.y) < 10:
            vel.y = 0
    
    # Rebond murs
    if pos.x - rayon < 0:
        pos.x = rayon
        vel.x = abs(vel.x) * REBOND
    elif pos.x + rayon > LARGEUR:
        pos.x = LARGEUR - rayon
        vel.x = -abs(vel.x) * REBOND
    
    # Affichage
    ecran.fill(NOIR)
    pygame.draw.line(ecran, GRIS, (0, SOL), (LARGEUR, SOL), 2)
    pygame.draw.circle(ecran, ROUGE, (int(pos.x), int(pos.y)), rayon)
    
    hauteur = SOL - pos.y - rayon
    info1 = font.render(f"Vélocité: ({vel.x:.0f}, {vel.y:.0f})", True, BLANC)
    info2 = font.render(f"Hauteur: {hauteur:.0f} px", True, BLANC)
    info3 = font.render("Clic: repositionner", True, GRIS)
    ecran.blit(info1, (10, 10))
    ecran.blit(info2, (10, 42))
    ecran.blit(info3, (10, HAUTEUR - 30))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 2

In [ ]:
%%writefile solution_ex2_platformer.py
import pygame
import sys
from pygame.math import Vector2

pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 2 : Mini-platformer")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
BLEU = (80, 120, 255)
VERT = (80, 200, 80)
MARRON = (139, 90, 43)

# Constantes physiques
GRAVITE = 1200
FORCE_SAUT = -500
ACCEL = 800
VITESSE_MAX = 300
FRICTION = 0.85
MAX_SAUTS = 2

# Joueur
pos = Vector2(100, 400)
vel = Vector2(0, 0)
TAILLE = (30, 40)
au_sol = False
nb_sauts = 0

# Plateformes (x, y, largeur, hauteur)
plateformes = [
    pygame.Rect(0, HAUTEUR - 40, LARGEUR, 40),     # Sol
    pygame.Rect(150, 420, 200, 20),                  # Plateforme basse
    pygame.Rect(400, 320, 200, 20),                  # Plateforme moyenne
    pygame.Rect(200, 220, 200, 20),                  # Plateforme haute
]

running = True
while running:
    dt = clock.tick(60) / 1000.0
    dt = min(dt, 0.05)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and nb_sauts < MAX_SAUTS:
                vel.y = FORCE_SAUT
                nb_sauts += 1
                au_sol = False
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE and vel.y < 0:
                vel.y *= 0.5
    
    # Mouvement horizontal
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        vel.x -= ACCEL * dt
    elif keys[pygame.K_RIGHT]:
        vel.x += ACCEL * dt
    else:
        vel.x *= FRICTION
    
    vel.x = max(-VITESSE_MAX, min(VITESSE_MAX, vel.x))
    
    # Gravité
    vel.y += GRAVITE * dt
    
    # Déplacer horizontalement
    pos.x += vel.x * dt
    rect_joueur = pygame.Rect(pos.x, pos.y, *TAILLE)
    
    # Collision horizontale
    for plat in plateformes:
        if rect_joueur.colliderect(plat):
            if vel.x > 0:
                pos.x = plat.left - TAILLE[0]
            elif vel.x < 0:
                pos.x = plat.right
            vel.x = 0
    
    # Déplacer verticalement
    pos.y += vel.y * dt
    rect_joueur = pygame.Rect(pos.x, pos.y, *TAILLE)
    
    au_sol = False
    for plat in plateformes:
        if rect_joueur.colliderect(plat):
            if vel.y > 0:  # Tombe
                pos.y = plat.top - TAILLE[1]
                vel.y = 0
                au_sol = True
                nb_sauts = 0
            elif vel.y < 0:  # Monte
                pos.y = plat.bottom
                vel.y = 0
    
    # Limiter aux bords
    pos.x = max(0, min(pos.x, LARGEUR - TAILLE[0]))
    
    # Affichage
    ecran.fill(NOIR)
    
    # Plateformes
    for i, plat in enumerate(plateformes):
        couleur = VERT if i == 0 else MARRON
        pygame.draw.rect(ecran, couleur, plat)
    
    # Joueur
    pygame.draw.rect(ecran, BLEU, (int(pos.x), int(pos.y), *TAILLE))
    
    # Info
    info = font.render(f"Sauts: {nb_sauts}/{MAX_SAUTS} | Au sol: {au_sol}", True, BLANC)
    ctrl = font.render("Flèches: bouger | ESPACE: sauter (x2)", True, (150, 150, 150))
    ecran.blit(info, (10, 10))
    ecran.blit(ctrl, (10, HAUTEUR - 30))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 3

In [ ]:
%%writefile solution_ex3_easing.py
import pygame
import sys
import math

pygame.init()

LARGEUR, HAUTEUR = 900, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 3 : Fonctions d'easing")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (80, 80, 80)
ROUGE = (255, 80, 80)
VERT = (80, 255, 80)


# Fonctions d'easing
def linear(t):
    return t

def ease_in_quad(t):
    return t * t

def ease_out_quad(t):
    return 1 - (1 - t) ** 2

def ease_out_bounce(t):
    if t < 1/2.75:
        return 7.5625 * t * t
    elif t < 2/2.75:
        t -= 1.5/2.75
        return 7.5625 * t * t + 0.75
    elif t < 2.5/2.75:
        t -= 2.25/2.75
        return 7.5625 * t * t + 0.9375
    else:
        t -= 2.625/2.75
        return 7.5625 * t * t + 0.984375

def ease_out_elastic(t):
    if t == 0 or t == 1:
        return t
    return 2 ** (-10 * t) * math.sin((t - 0.075) * (2 * math.pi) / 0.3) + 1


def lerp(a, b, t):
    return a + (b - a) * t


easings = [
    ("Linear", linear),
    ("Ease In (Quad)", ease_in_quad),
    ("Ease Out (Quad)", ease_out_quad),
    ("Bounce", ease_out_bounce),
    ("Elastic", ease_out_elastic),
]

index_easing = 0
duree_anim = 2.0  # secondes
temps = 0
en_cours = True

# Points A et B
A_x, A_y = 100, 200
B_x, B_y = 750, 200
taille_carre = 40

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_SPACE:
                temps = 0
                en_cours = True
            # Changer l'easing
            for i in range(5):
                if event.key == pygame.K_1 + i:
                    index_easing = i
                    temps = 0
                    en_cours = True
    
    # Avancer l'animation
    if en_cours:
        temps += dt
        if temps >= duree_anim:
            temps = duree_anim
            en_cours = False
    
    # Calculer la position avec easing
    t_brut = min(temps / duree_anim, 1.0)
    nom_easing, func_easing = easings[index_easing]
    t_ease = func_easing(t_brut)
    
    pos_x = lerp(A_x, B_x, t_ease)
    pos_y = A_y
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Titre
    titre = font.render(f"Easing: {nom_easing}", True, BLANC)
    ecran.blit(titre, (LARGEUR // 2 - titre.get_width() // 2, 20))
    
    # Points A et B
    pygame.draw.circle(ecran, VERT, (A_x, A_y), 8)
    pygame.draw.circle(ecran, ROUGE, (B_x, B_y), 8)
    pygame.draw.line(ecran, GRIS, (A_x, A_y), (B_x, B_y), 1)
    
    # Carré animé
    pygame.draw.rect(ecran, (80, 150, 255),
                    (int(pos_x) - taille_carre // 2,
                     int(pos_y) - taille_carre // 2,
                     taille_carre, taille_carre))
    
    # Progression
    prog_txt = font.render(f"t = {t_brut:.2f} -> eased = {t_ease:.2f}", True, BLANC)
    ecran.blit(prog_txt, (LARGEUR // 2 - prog_txt.get_width() // 2, 270))
    
    # Courbe d'easing
    courbe_x = 200
    courbe_y = 350
    courbe_w = 500
    courbe_h = 180
    
    # Fond de la courbe
    pygame.draw.rect(ecran, (20, 20, 30), (courbe_x, courbe_y, courbe_w, courbe_h))
    pygame.draw.rect(ecran, GRIS, (courbe_x, courbe_y, courbe_w, courbe_h), 1)
    
    # Dessiner la courbe
    nb_points = 100
    points = []
    for i in range(nb_points + 1):
        t_i = i / nb_points
        v = func_easing(t_i)
        px = courbe_x + t_i * courbe_w
        py = courbe_y + courbe_h - v * courbe_h
        points.append((px, py))
    
    if len(points) > 1:
        pygame.draw.lines(ecran, (100, 200, 255), False, points, 2)
    
    # Point courant sur la courbe
    px_courant = courbe_x + t_brut * courbe_w
    py_courant = courbe_y + courbe_h - t_ease * courbe_h
    pygame.draw.circle(ecran, ROUGE, (int(px_courant), int(py_courant)), 6)
    
    # Axes
    ecran.blit(font.render("0", True, GRIS), (courbe_x - 15, courbe_y + courbe_h - 10))
    ecran.blit(font.render("1", True, GRIS), (courbe_x + courbe_w + 5, courbe_y - 5))
    ecran.blit(font.render("t", True, GRIS), (courbe_x + courbe_w // 2, courbe_y + courbe_h + 5))
    
    # Instructions
    instructions = []
    for i, (nom, _) in enumerate(easings):
        marker = " <" if i == index_easing else ""
        instructions.append(f"{i+1}: {nom}{marker}")
    
    y_inst = 350
    for txt in instructions:
        ecran.blit(font.render(txt, True, (150, 150, 150)), (10, y_inst))
        y_inst += 25
    
    ctrl = font.render("ESPACE: relancer", True, (100, 100, 100))
    ecran.blit(ctrl, (10, HAUTEUR - 30))
    
    pygame.display.flip()

pygame.quit()
sys.exit()